<a href="https://colab.research.google.com/github/JonMcEntee/hm-fashion-recommendations/blob/main/H%26M_Fashion_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [1]:
# Mount into drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# H&M Fashion Recommendations

This notebook details my approach to Kaggle's [H&M Personalized Fashion Recommendations](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/overview) competition, focused on predicting customers' next fashion purchases. With vast product selections becoming standard in retail, delivering precisely tailored suggestions is key to enhancing shopping experiences and brand loyalty in modern fashion commerce.

In [ ]:
articles = pd.read_csv("./drive/My Drive/h-and-m-personalized-fashion-recommendations/articles.csv")
customers = pd.read_csv("./drive/My Drive/h-and-m-personalized-fashion-recommendations/customers.csv")
transactions = pd.read_csv("./drive/My Drive/h-and-m-personalized-fashion-recommendations/transactions_train.csv")